# Server OLED Display Controller

This notebook displays a parking lot counter on an OLED display. It communicates with `/server` using HTTP, updating the counters every 2 seconds.

First we must import some stuff and load `config.json`. This is used mainly to store the server address for us.

In [ ]:
from pynq.lib.pmod import *
from pynq import PL

import oled
import json
import requests
import time

# load config
config = json.load(open('config.json'))
config

Now we instantiate the `OledDisplay` class found in `oled.py`. It is mostly a clone of `~/base/pmod/pmod_grove_oled.ipynb`.

We also get the light bar, reset the display, and clear the display.

In [ ]:
# display lib from oled.py
show_oled = config.get('show_oled')
if show_oled:
    display = oled.OledDisplay(PMODA, PMOD_GROVE_G3)
    
    # reset diplay
    PL.reset()
    display.clear()

# get light bar if config requests it
show_bar = config.get('show_bar')
if show_bar:
    bar = Grove_LEDbar(PMODB, PMOD_GROVE_G3)

Next we define the `write` function, which simply prints a string on a line of the OLED display. It keeps track of the current length of each line, so that when a shorter line replaces a longer one, it can clear out the trailing characters.

In [ ]:
# track line lengths
old_lines = [0 for _ in range(8)]

# easy printing
def write(text, line=0):
    global display
    global old_lines
    
    old_len = old_lines[line]
    # prevent overflow
    if len(text) > 16:
        text = text[:16]
    new_len = len(text)

    # write the text to the line
    display.set_XY(line, 0)
    display.write(text)
    
    # clear trailing chars
    if new_len < old_len:
        display.set_XY(line, new_len)
        display.write(' ' * (old_len - new_len))
    
    # save the new line length
    old_lines[line] = new_len

Now we connect to the server and update the display every 2 seconds. We iterate through the first 4 (or fewer) lots, printing their names and counts to the display. Finally the display object is deleted, cleaning things up.

In [ ]:
server = config['server']
while True:
    res = requests.get('{}/lots'.format(server)).json()
    lots = res['lots']
    
    # send to light bar
    if show_bar:
        total = sum(min(max(l['cars'], 0), l['capacity']) for l in lots)
        capacity = sum(l['capacity'] for l in lots)
        bar.write_level(int(10 * total / capacity), 2, 1)
    
    if show_oled:
        # show detailed text
        for i in range(4):
            if i < len(lots):
                lot = lots[i]
                write('{}: {}'.format(lot['name'], lot['cars']), i * 2)
            else:
                write('', i * 2)
    
    time.sleep(2)

if show_oled:
    del display